# 1. 原始方法

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

template = ChatPromptTemplate([
    ("system", """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation: """),
    ("placeholder", "{history}"),
    ("user", "{input}"),
    ("ai", "")
])

chat = ChatOpenAI(model_name="openai/gpt-4o-mini")
chain = template | chat | StrOutputParser()

resp = chain.invoke({
    "history": [
        ("human", "我周五晚上要去看电影"),
    ],
    "input": "我周五要去做什么"
})

print(resp)


你提到你周五晚上要去看电影，所以可以推测你周五会去电影院享受一部电影！你已经决定看哪部电影了吗？或者你是跟朋友一起去吗？


# 通过 ConversationChain 实现

**注：**

**该方式是以前的调用方法，目前已经非废弃，只是为了保证 Langchain 的向下兼容，依然提供。**

**现在推荐通过 `RunnableWithMessageHistory` 来实现**

In [15]:
# Deprecated
from langchain.chains.conversation.base import ConversationChain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="openai/gpt-4o-mini")
conversation = ConversationChain(llm=llm)
resp = conversation.invoke({"input": "我周五晚上要去看电影"})
print(resp.get("response"), "\n\n", conversation.memory, "\n\n\n")
resp = conversation.invoke({"input": "我周五晚上要做什么？"})
print(resp.get("response"), "\n\n", conversation.memory)



那听起来很不错！你打算看哪部电影呢？最近有很多新片上映，比如《沙丘：第二部》和《黑豹：瓦坎达永存》。如果你喜欢科幻或者超级英雄电影，这些都是不错的选择！或者，你更喜欢其他类型的电影，比如喜剧或爱情片？ 

 chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='我周五晚上要去看电影', additional_kwargs={}, response_metadata={}), AIMessage(content='那听起来很不错！你打算看哪部电影呢？最近有很多新片上映，比如《沙丘：第二部》和《黑豹：瓦坎达永存》。如果你喜欢科幻或者超级英雄电影，这些都是不错的选择！或者，你更喜欢其他类型的电影，比如喜剧或爱情片？', additional_kwargs={}, response_metadata={})]) 



你周五晚上计划去看电影，所以可以期待一个愉快的夜晚！看完电影后，你可能会想去吃点东西或者和朋友聊聊电影的情节和角色。你有没有想过看完电影后要去哪里？有些人喜欢去咖啡馆或餐厅放松一下。你有什么特别的计划吗？ 

 chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='我周五晚上要去看电影', additional_kwargs={}, response_metadata={}), AIMessage(content='那听起来很不错！你打算看哪部电影呢？最近有很多新片上映，比如《沙丘：第二部》和《黑豹：瓦坎达永存》。如果你喜欢科幻或者超级英雄电影，这些都是不错的选择！或者，你更喜欢其他类型的电影，比如喜剧或爱情片？', additional_kwargs={}, response_metadata={}), HumanMessage(content='我周五晚上要做什么？', additional_kwargs={}, response_metadata={}), AIMessage(content='你周五晚上计划去看电影，所以可以期待一个愉快的夜晚！看完电影后，你可能会想去吃点东西或者和朋友聊聊电影的情节和角色。你有没有想过看完电影后要去哪里？有些人喜欢去咖啡馆或餐厅放松一下。你有什

# 通过 `RunnableWithMessageHistory` 来实现

In [32]:
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model_name="openai/gpt-4o-mini")

store = {}


def get_session_with_history(session_id: str) -> BaseChatMessageHistory:
    return store.setdefault(session_id, InMemoryChatMessageHistory())


chat_with_message_history = RunnableWithMessageHistory(chat_model,
                                                       get_session_with_history)

config = {"configurable": {"session_id": "example1"}}

resp = chat_with_message_history.invoke(input="我周五晚上准备去看电影", config=config)
print(resp.content, "\n")
resp = chat_with_message_history.invoke(input="我之前说我周五晚上准备去干什么", config=config)
print(resp.content)

听起来很不错！你打算看什么电影呢？或者你已经有了想看的片子吗？ 

你提到过你准备去看电影。你有没有决定好要看的电影或者影院呢？


# 配合 ChatPromptTemplate 使用

In [58]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all questions to the best of your ability.",),
    ("placeholder", "{history}"),
    ("human", "{user_input}",)
])

chat_model = ChatOpenAI(model_name="openai/gpt-4o-mini")

store = {}

chat_with_message_history = RunnableWithMessageHistory(
    prompt | chat_model,
    lambda session_id: store.setdefault(session_id, InMemoryChatMessageHistory()),
    # input_messages_key="user_input",
    # history_messages_key="chat_history",
)
config = {"configurable": {"session_id": "example3"}}
resp = chat_with_message_history.invoke(input={"user_input": "我周五晚上准备去看电影"}, config=config)
print(resp.content, "\n")
resp = chat_with_message_history.invoke(input={"user_input": "我之前说我周五晚上准备去干什么"}, config=config)
print(resp.content)

听起来不错！你打算看哪部电影呢？ 

看起来你之前提到过你周五晚上计划去看电影。你决定好要看哪部电影了吗？


In [51]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all questions to the best of your ability.",),
    ("placeholder", "{messages1}"),
])

chat_model = ChatOpenAI(model_name="openai/gpt-4o-mini")

store = {}

chat_with_message_history = RunnableWithMessageHistory(
    prompt | chat_model,
    lambda session_id: store.setdefault(session_id, InMemoryChatMessageHistory()),
    input_messages_key="messages1", # ChatPromptTemplate 传入的是对话列表 list，这里需要声明 key
)
config = {"configurable": {"session_id": "example3"}}
resp = chat_with_message_history.invoke(input={"messages1": [HumanMessage(content="我周五晚上准备去看电影")]}, config=config)
print(resp.content, "\n")
# resp = chat_with_message_history.invoke(input={"messages": [HumanMessage(content="我之前说我周五晚上准备去干什么")]}, config=config)
resp = chat_with_message_history.invoke(input={"messages1": [("user", "我之前说我周五晚上准备去干什么")]}, config=config)
print(resp.content)

听起来很不错！你打算看什么电影呢？有什么特别想看的类型吗？ 

你提到你准备去看电影。你已经决定好要看哪部电影了吗？或者你有什么想看的电影推荐吗？


In [55]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all questions to the best of your ability.",),
    MessagesPlaceholder(variable_name="messages2"),
])

chat_model = ChatOpenAI(model_name="openai/gpt-4o-mini")

store = {}

chat_with_message_history = RunnableWithMessageHistory(
    prompt | chat_model,
    lambda session_id: store.setdefault(session_id, InMemoryChatMessageHistory()),
    # input_messages_key="messages2", # Template 的这种写法，MessagesPlaceholder(variable_name="messages2")，这里无需再声明
)
config = {"configurable": {"session_id": "example3"}}
resp = chat_with_message_history.invoke(input={"messages2": [HumanMessage(content="我周五晚上准备去看电影")]}, config=config)
print(resp.content, "\n")
resp = chat_with_message_history.invoke(input={"messages2": [("user", "我之前说我周五晚上准备去干什么")]}, config=config)
print(resp.content)

听起来不错！你已经选好要看的电影了吗？或者你有什么类型的电影特别感兴趣？ 

你之前提到你准备去看电影。如果你需要推荐电影或是讨论一下具体的计划，随时告诉我！


In [56]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all questions to the best of your ability.",),
    MessagesPlaceholder(variable_name="messages2"),
])

chat_model = ChatOpenAI(model_name="openai/gpt-4o-mini")

store = {}

chat_with_message_history = RunnableWithMessageHistory(
    prompt | chat_model,
    lambda session_id: store.setdefault(session_id, InMemoryChatMessageHistory()),
)
config = {"configurable": {"session_id": "example3"}}
resp = chat_with_message_history.invoke(input={"messages2": HumanMessage(content="我周五晚上准备去看电影")}, config=config)
print(resp.content, "\n")
resp = chat_with_message_history.invoke(input={"messages2": ("user", "我之前说我周五晚上准备去干什么")}, config=config)
print(resp.content)

那听起来很不错！你已经决定好要看哪部电影了吗？或者你在考虑哪种类型的电影？ 

你之前提到你周五晚上准备去看电影。你已经决定好要看哪部电影了吗？或者有想要一起去的人吗？


In [24]:
history = store.get("example3")
# print(store)
assert isinstance(history, InMemoryChatMessageHistory)
# history.messages
print(len(history.messages))
print(history.messages)

4
[HumanMessage(content='我周五晚上准备去看电影', additional_kwargs={}, response_metadata={}), AIMessage(content='那听起来很不错！你已经决定要看哪部电影了吗？或者你有什么类型的电影偏好吗？', additional_kwargs={'refusal': ''}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 35, 'total_tokens': 61, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'openai/gpt-4o-mini', 'system_fingerprint': 'fp_d02d531b47', 'finish_reason': 'stop', 'logprobs': None}, id='run-ac9dd930-44c0-42dc-98a3-e171bab6b23d-0', usage_metadata={'input_tokens': 35, 'output_tokens': 26, 'total_tokens': 61, 'input_token_details': {}, 'output_token_details': {}}), ('user', '我之前说我周五晚上准备去干什么'), AIMessage(content='是的，你提到你周五晚上准备去看电影。你对这部电影有什么期待吗，或者你已经选好想看的电影了吗？', additional_kwargs={'refusal': ''}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 79, 'total_tokens': 113, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'openai/gpt-4o-mini', 'system_fingerp

# 控制历史记录的大小

https://python.langchain.com/v0.2/docs/how_to/chatbots_memory/

In [6]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all questions to the best of your ability.",),
    MessagesPlaceholder(variable_name="messages"),
])

chat_model = ChatOpenAI(model_name="openai/gpt-4o-mini")

history = InMemoryChatMessageHistory()

chat_with_message_history = RunnableWithMessageHistory(
    prompt | chat_model,
    lambda session_id: history,
)
config = {"configurable": {"session_id": "example3"}}
resp = chat_with_message_history.invoke(input={"messages": [HumanMessage(content="我周五晚上准备去看电影")]}, config=config)
print(resp.content, "\n")
resp = chat_with_message_history.invoke(input={"messages": [HumanMessage(content="我之前说我周五晚上准备去干什么")]}, config=config)
print(resp.content)

那听起来很不错！你已经决定看哪部电影了吗？如果需要推荐，可以告诉我你喜欢的类型或者演员，我可以帮你推荐一些不错的选择。 

你之前提到你周五晚上准备去看电影。如果你有其他的计划或者想聊聊电影的选择，随时告诉我！


In [8]:
print(len(history.messages))
for msg in history.messages:
    print(msg.content)

4
我周五晚上准备去看电影
那听起来很不错！你已经决定看哪部电影了吗？如果需要推荐，可以告诉我你喜欢的类型或者演员，我可以帮你推荐一些不错的选择。
我之前说我周五晚上准备去干什么
你之前提到你周五晚上准备去看电影。如果你有其他的计划或者想聊聊电影的选择，随时告诉我！


In [39]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage, trim_messages
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all questions to the best of your ability.",),
    MessagesPlaceholder(variable_name="messages"),
])

chat_model = ChatOpenAI(model_name="openai/gpt-4o-mini")

history = InMemoryChatMessageHistory()

trimmer = trim_messages(max_tokens= 3, token_counter=len, strategy="first")

chat_with_message_history = RunnableWithMessageHistory(
    trimmer | prompt | chat_model,
    lambda session_id: history,
)
config = {"configurable": {"session_id": "example3"}}
resp = chat_with_message_history.invoke(input={"messages": [HumanMessage(content="我周五晚上准备去看电影")]}, config=config)
print(resp.content, "\n")
resp = chat_with_message_history.invoke(input={"messages": [HumanMessage(content="我之前说我周五晚上准备去干什么？")]}, config=config)
print(resp.content)
resp = chat_with_message_history.invoke(input={"messages": [HumanMessage(content="我之前说我周五晚上准备去干什么？")]}, config=config)
print(resp.content)
resp = chat_with_message_history.invoke(input={"messages": [HumanMessage(content="我之前说我周五晚上准备去干什么？")]}, config=config)
print(resp.content)

那听起来不错！你打算看哪部电影呢？或者你在找推荐吗？ 

你之前提到你准备去看电影。你已经选好电影了吗？
你说你准备去看电影。你已经选好电影了吗？
你之前提到你准备去看电影。如果你有特定的电影或者类型想要讨论，欢迎分享！


In [40]:
print(len(history.messages))
for msg in history.messages:
    print(msg.content)

8
我周五晚上准备去看电影
那听起来不错！你打算看哪部电影呢？或者你在找推荐吗？
我之前说我周五晚上准备去干什么？
你之前提到你准备去看电影。你已经选好电影了吗？
我之前说我周五晚上准备去干什么？
你说你准备去看电影。你已经选好电影了吗？
我之前说我周五晚上准备去干什么？
你之前提到你准备去看电影。如果你有特定的电影或者类型想要讨论，欢迎分享！


# Todo

把基于内存的历史换成下面的：

- FileChatMessageHistory
- RedisChatMessageHistory

```py
from langchain_community.chat_message_histories import FileChatMessageHistory
```

如何有效管理 session

